In [13]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import backbone_utils
import os
from PIL import Image
import torch
from torchvision import transforms as transforms
import numpy as np

In [14]:
backbone = torchvision.models.mobilenet_v2(pretrained=False).features
backbone.out_channels = 1280

In [15]:
# anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
#                                    aspect_ratios=((0.5, 1.0, 2.0),))

In [16]:
# roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
#                                                 output_size=7,
#                                                 sampling_ratio=2)

In [17]:
model = FasterRCNN(backbone,
                   num_classes=2)

In [18]:
model.cuda()

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm

In [19]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm

In [20]:
transforms = torchvision.transforms.Compose([
    transforms.Resize((921, 1081)),
    transforms.CenterCrop((920, 1080)),
    transforms.ToTensor()
])
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

In [21]:
dir_files = []
for x in os.listdir("dataset/test"):
    dir_files.append(
        os.path.join("dataset/test", x))

In [23]:
pred_mobileNet = []
mean_time = []

model.cuda()
model.eval()
with torch.no_grad():
    for file in dir_files:
        img = Image.open(file)
        img = transforms(img).unsqueeze(0)
        img = img.cuda()
        start.record()
        pred_mobileNet.append(model(img))
        end.record()
        torch.cuda.synchronize()

        print(start.elapsed_time(end))
        mean_time.append(start.elapsed_time(end))
print("MEAN: ", np.mean(mean_time))

665.0344848632812
665.0363159179688
667.9593505859375
664.7700805664062
666.188720703125
MEAN:  665.7977905273438


In [24]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(num_classes=2)

In [26]:
pred_resNet50 = []
mean_time = []
model.cuda()
model.eval()
with torch.no_grad():
    for file in dir_files:
        img = Image.open(file)
        img = transforms(img).unsqueeze(0)
        img = img.cuda()
        start.record()
        pred_resNet50.append(model(img))
        end.record()
        torch.cuda.synchronize()

        print(start.elapsed_time(end))
        mean_time.append(start.elapsed_time(end))
        
print("MEAN: ", np.mean(mean_time))

934.1403198242188
936.9476928710938
952.4248046875
945.0905151367188
943.8165283203125
MEAN:  942.4839721679688


In [31]:
pred_mobileNet[0][0]['labels']

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1], device='cuda:0')

In [32]:
pred_resNet50[0][0]['labels']

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1], device='cuda:0')